In [ ]:
import toml
import glob
import pandas as pd
import os
import sys
import numpy as np

In [ ]:
import test_compute_fss_lib
import test_multi_fss_lib
import test_compute_fss
import test_multi_fss_proc

In [ ]:
obs = 0

SAT_FOLDER = '/work/asc/machine_learning/projects/iMagine/Medslik-II-medslik_II_3_01_JUNO/MEDSLIKII_workflow_/observations_'
DAYS_GROUP = '/2021082*'
MEDSLIKII_OUT_DIR = '/work/asc/machine_learning/projects/iMagine/bayes_opt_workflow/test/OUT/MDK_SIM_2021_08_23_1000_syria'

OUT_FOLDER = '/work/asc/machine_learning/projects/iMagine/bayes_opt_workflow/test/MDKII_res/MDKII_'
DETECTION_FOLDER = OUT_FOLDER + '/detection_' + str(obs)

In [ ]:
''' Punto in cui agire per restringere le osservazioni a pochi giorni '''
list_of_obs = glob.glob(SAT_FOLDER + DAYS_GROUP)

In [ ]:
# get sim_length
sim_lenght = config['medslik2']['sim_extent']['sim_length']

# get simulation date
# day
dd = config['medslik2']['sim_date']['day']
# month
mm = config['medslik2']['sim_date']['month']
# year
yy = config['medslik2']['sim_date']['year']
# hour
hh = config['medslik2']['sim_date']['hour']

sim_date = test_multi_fss_lib.get_sim_date(yy, mm, dd, hh)
print(sim_date)

fss_df = pd.DataFrame(columns=['folder', 'fss'])

In [ ]:
for slick_folder in list_of_obs:
    
    null, slick_id=os.path.split(slick_folder)
    
    slick_date = test_multi_fss_lib.get_slick_date(slick_id)

    if (slick_date-sim_date) < float(sim_lenght)/24:
        
        #out_folder, detection_folder = test_multi_fss_lib.set_out_folder(parent, obs)

        test_compute_fss.compute_single_fss(MEDSLIKII_OUT_DIR, slick_folder, DETECTION_FOLDER)
        
        #pysteps_string = 'python ' + Path.SINGLE_DET_FSS
        #args_string = '-f ' + Path.MEDSLIKII_OUT_DIR + ' -o ' + slick_folder + ' -u ' + detection_folder + ' 2> /dev/null'
        #os.system(pysteps_string + args_string)

        files = glob.glob(DETECTION_FOLDER + '/fss_syria_*.txt')

        for file in files:

            f = pd.read_csv(file, sep = ' ', header = None)

            row = pd.DataFrame([{"folder" : OUT_FOLDER, "fss" : round(f.iloc[0,1], 4)}])
            fss_df = pd.concat([fss_df, row])

        obs += 1

FSS = fss_df['fss'].max()

In [19]:
import netCDF4
import numpy as np

def get_surface_parcels(fname,time_index):

    f = netCDF4.Dataset(fname)
    
    lat = np.longdouble(f.variables['latitude'][time_index,:])
    lon = np.longdouble(f.variables['longitude'][time_index,:])
    
    return lat, lon

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [20]:
import os

import sys
sys.path.append("/work/asc/machine_learning/projects/iMagine/bayes_opt")

from src.controller.PathController import PathController
from compute_fss.test_compute_fss_lib import get_mdksim_date, get_obs_date, get_lonlat_minmax

path_controller_instance = PathController("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml")
#print(path_controller_instance.get_MEDSLIK_OUT_DIR())
fname = path_controller_instance.get_MEDSLIK_OUT_DIR() + '/spill_properties.nc'

simulation_date = get_mdksim_date(path_controller_instance.get_MEDSLIK_OUT_DIR())
observation_date = get_obs_date("/work/asc/machine_learning/projects/iMagine/bayes_opt/use_case_observations/syria/observations/20210824-0343-SYR-PL-B-01-S1")

time_index = find_nearest(simulation_date, observation_date)

lons_f, lats_f = get_surface_parcels(fname, time_index)
lonmin, latmin, lonmax, latmax = get_lonlat_minmax(lons_f, lats_f)

print("lat:", lonmin, latmin)
print("lon:", lonmax, latmax)

CIAO2
print: 35.17259979248046875 35.85400390625 35.47209930419921875 35.95107269287109375
lat: 35.17259979248046875 35.85400390625
lon: 35.47209930419921875 35.95107269287109375
